In [1]:
!pip install --upgrade open_spiel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 2.7 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.special import expit
import pyspiel
import enum

In [3]:
#actions = ['lead_' + str(i * 0.5) + '_steal' for i in range(41)] + ['lead_' + str(i * 0.5) + '_stay' for i in range(41)] + ['pitch', 'pickoff']
actions = ['lead_' + str(i) + '_steal' for i in range(10,13)] + ['lead_' + str(i) + '_stay' for i in range(10,13)] + ['pitch', 'pickoff']
action_dict = {actions[i]: i for i in range(len(actions))}

Action = enum.IntEnum('Action', action_dict)

_GAME_TYPE = pyspiel.GameType(
    short_name="sb_ab",
    long_name="One AB Stolen Base Game",
    dynamics=pyspiel.GameType.Dynamics.SEQUENTIAL,
    chance_mode=pyspiel.GameType.ChanceMode.EXPLICIT_STOCHASTIC,
    information=pyspiel.GameType.Information.IMPERFECT_INFORMATION,
    utility=pyspiel.GameType.Utility.ZERO_SUM,
    reward_model=pyspiel.GameType.RewardModel.TERMINAL,
    max_num_players=2,
    min_num_players=2,
    provides_information_state_string=True,
    provides_information_state_tensor=False, #at some point I may want this to be true
    #I don't think you need to provide both information states and observation states I think you can just do info states
    provides_observation_string=False,
    provides_observation_tensor=False,
    provides_factored_observation_string=False)
_GAME_INFO = pyspiel.GameInfo(
    num_distinct_actions=len(Action),
    max_chance_outcomes=3,
    num_players=2,
    min_utility=-1.,
    max_utility=1.,
    utility_sum=0.0,
    max_game_length=18) #6 pitches/leads = 12 plus 3 pickoffs/leads = 6 so 18 total

class SBABGame(pyspiel.Game):

  def __init__(self, params=None):
    super().__init__(_GAME_TYPE, _GAME_INFO, params or dict())

  def new_initial_state(self):
    """Returns a state corresponding to the start of a game."""
    return SBABState(self)

  def make_py_observer(self, iig_obs_type = None, params = None):
    return SBABObserver()



class SBABState(pyspiel.State):

  def __init__(self, game):
    """Constructor; should only be called by Game.new_initial_state."""
    super().__init__(game)
    self.b1 = 1
    self.b2 = 0
    self.b3 = 0
    self.outs = 0
    self.balls = 0
    self.strikes = 0
    self.disengagements = 0
    self.history = []
    self.lead_dist = 0
    self._game_over = False
    self._next_player = 0

  # OpenSpiel (PySpiel) API functions are below. This is the standard set that
  # should be implemented by every sequential-move game with chance.

  def current_player(self):
    """Returns id of the next player to move, or TERMINAL if game is over."""
    if self._game_over:
      return pyspiel.PlayerId.TERMINAL
    if len(self.history) > 0:
      if self.history[-1] in {'pitch', 'pickoff'}:
        return pyspiel.PlayerId.CHANCE
    return self._next_player

  def _legal_actions(self, player):
    """Returns a list of legal actions, sorted in ascending order."""
    assert player >= 0
    if player == 0: #baserunner
      return [Action[i] for i in actions[:-2]]
    else: #pitcher
      return [Action['pitch'], Action['pickoff']]

  def chance_outcomes(self):
    """Returns the possible chance outcomes and their probabilities."""
    assert self.is_chance_node()
    if self.history[-1] == 'pickoff':
      success_prob = expit(0.3 * self.lead_dist - 5)
      #0 means out, 1 means safe
      return [(0, success_prob), (1, 1-success_prob)]

    if self.history[-1] == 'pitch':
      if 'steal' in self.history[-2]:
        #in this history the batter chose to steal and the pitcher pitched so we had a stolen base attempt
        sb_prob = expit(0.5 * self.lead_dist - 5)
        return [(0, 1-sb_prob), (1, sb_prob)]

      else:
        #no stolen base attempt, just a pitch. 2 is ball, 3 is strike, 4 is bip
        return [(2, 1/3), (3, 1/3), (4, 1/3)]

  def _apply_action(self, action):
    """Applies the specified action to the state."""
    if self.is_chance_node():
      if self.history[-1] == 'pickoff':
        #pickoff attempt
        self.disengagements += 1
        if action == 0:
          #baserunner is out
          self._game_over = True
        if action == 1 and self.disengagements == 3:
          #three disengagements without a succesful pickoff
          self._game_over = True

      else: #the last action was a pitch
        if 'steal' in self.history[-2]:
          #the baserunner attempted a steal on the pitch
          self._game_over = True
        else:
          if action == 2:
            #called ball
            self.balls += 1
            if self.balls == 4:
              self._game_over = True
          elif action == 3:
            #strike
            self.strikes += 1
            if self.strikes == 3:
              self._game_over = True
          else:
            #bip
            self._game_over = True

      self.history.append(str(action))
      self._next_player = 0

    else:
      action_str = actions[action]
      self.history.append(action_str)
      if action_str in {'pitch', 'pickoff'}:
        #pitch or pickoff, so this was a pitcher action
        self._next_player = 0
      else:
        self._next_player = 1

    ##TESTING PURPOSES
    #end the game after some number of pitches/disengagements
    if self.balls + self.strikes + self.disengagements >= 8:
      self._game_over = True

  def _action_to_string(self, player, action):
    """Action -> string."""
    if player == pyspiel.PlayerId.CHANCE:
      if action == 0:
        return 'Out'
      elif action == 1:
        return 'Safe'
      elif action == 2:
        return 'Ball'
      elif action == 3:
        return 'Strike'
      elif action == 4:
        return 'Batted Ball'

    else:
      return actions[Action[action]]

  def is_terminal(self):
    """Returns True if the game is over."""
    return self._game_over

  def returns(self):
    """Total reward for each player over the course of the game so far."""
    if not self._game_over:
      return [0.,0.]

    if self.history[-1] in {'1', '2'}:
      return [1, -1]
    if self.history[-1] in {'0', '3'}:
      return [-1,1]
    if self.history[-1] == '4':
      return [-0.5, 0.5]

  def information_state_string(self):
    return '-'.join(self.history)


class SBABObserver:
  def __init__(self):
    self.tensor = []

  def set_from(self, state, player):
    pass

  def string_from(self, state, player):
    string = state.information_state_string()
    string = string.replace('stay', '')
    string = string.replace('steal', '')
    return string


pyspiel.register_game(_GAME_TYPE, SBABGame)

In [4]:
game = pyspiel.load_game('sb_ab')

In [ ]:
# state = game.new_initial_state()
# state.apply_action(Action['lead_10_steal'])
# state.apply_action(Action['pickoff'])
# state.apply_action(0)

In [ ]:
cfr_solver = pyspiel.CFRSolver(game)

In [ ]:
cfr_solver.tabular_average_policy().size()

2598952

In [5]:
#let's now try the same thing as the big cell below only this time we'll include the pickoff decision.
#We'll treat a pickoff or pitch decision as a stage of the game and we'll see how it gets bigger as the
#number of stages increases

#let n be the number of batter lead distances
n = len(actions[:-1])//2

#the first stage of the game has 1 root node plus n info sets, one for each batter lead
stage1 = n+1

#the second stage is the stage 1 prefixes times 3 which corresponds to the pickoff/safe history and
#the two nonterminal pitch histories: ball and strike and then times n for the batter leads again all added to the stage1 total
stage2 = stage1 + (stage1 * 3 * n)

#for stage 3 we again multiply the stage2 addition above stage 1 by 3 for the nonterminal pitch/pickoff outcomes
#then by n for the batter lead distances
stage3 = stage2 + ((stage2 - stage1) * 3 * n)


#now it starts to get tricky because we can't just repeat the same process as before
#because some histories are guaranteed to end before stage 4: namely any sequence
#where we got three strikes in a row or any sequence of 3 pickoffs in a row. The number
#So what we do is just like before we're going to take the stage3 total,
#we're going to add that to the product of the stage3 value above stage 2 times 3,
#but then before we multiply the lead distances we need to subtract
#2 * (n+1) * n * n, which is all the ways that we could have had three pickoffs
#or three strikes in the first three stages, then we can multiply by n
stage4 = stage3 + (((stage3 - stage2) * 3 - (2*(n+1)*n*n)) * n)

#stage 5 is similarly tricky because now not only do we have to worry about the strikeouts
#and three disengagements, but we also have to worry about four pitch walks.
#First, for strikeouts and three disengagements, there are 4 choose 3 ways that
#we can get a sequence of 3 strikes/dises in 4 pitches, but we need to subtract
#1 of those possibilities for the first three out of 4 option since we alread covered that.
#4 choose 3 is 4 so we have 3 options, but we need to double it for strikeouts and
#disengagements so 6. We then have to double it again because there are two alternatives
#for the one out of 4 outcomes in the sequence that aren't a strike/disengagement.
#for example for strikes that fourth stage could be a called ball or it could have
#been a disengagement, and for disengagements that fourth stage could have been a strike
#or a ball, so we double 6 to get 12 possibilities. Then we have 1 additional
#possiblity for four straight balls so 13 total.
#so we'll subtract 13 * (n+1)*n*n*n before multiplying by n again
stage5 = stage4 + (((stage4 - stage3) * 3 - (13 * (n+1)*n*n*n)) * n)

#At stage 6 we have 5 choose 3 ways to get a strikeout or balk on disengagements
#which is 10 total. We have to ignore the 4 ways where we get 3 in the first 4, which
#leaves just 6 of the original 10. And then in the other two pitches we could have
#either a ball or the nonstrike/nondisengagement outcome, so we have to multiply
#6 by 2*2 which gives us 24, and then double that since it's either a balk
#or a strikeout so 48 total. Then for walks on 5 pitches, we have 5 choose 4
#which is 5, and then we subtract 1 for the time that we get 4 balls in the first
#4 pitches, so 4, then we double that since the non ball could have been either a
#strike or a disengagement, so we ultimately subtract (48 + 8) * (n+1)*n**4 after multiplying by 3
#before multiplying by the final n
stage6 = stage5 + (((stage5-stage4) * 3 - (56 * (n+1)*n*n*n*n)) * n)


#At stage 7, we have 6 choose 3 ways to get a strikeout or balk, which is 20,
#but we have to ignore the 10 ways we can get to 3 in the first 5 stages,
#so that's 10. For the other three stages, we have two possibilities,
#so that's 10 * 2 * 2 * 2, which is 80. Then we double it since it's either a
#balk or strikeout so 160. However, we can't have three strikes and three disengagements
#in those 6 pitches. At least one of them has to be a ball, so we have to ignore
#the one way where all three bonus pitches are a strike/disengagement, and we have to
#do that twice, so we subtract 2 * 10 to get 140. Then for walks, we have 6 choose 4 which is 15 minus
#the 5 ways we could get a walk in the first 5 stages, so 10. Then multiply
#by 2 * 2 for the other two pitches, which is 40. So combining everything
#we have to subtract 180 * (n+1)*n**5 after multiplying by 3 and before multiplying by n again
stage7 = stage6 + (((stage6-stage5) * 3 - (180 * (n+1)*n**5)) * n)


#at stage 8, we have 7 choose 3 ways to get a strikeout or balk, which is 35,
#but we subtract the 20 ways we can get 3 in the first 6, so 15. The other four stages
#could be two outcomes, so 15 * 2 * 2* 2* 2  which is 240. However, they can't all
#be balls or else we'd have a walk, so we have to subtract 15 for that scenario which
#gives 225. We also have to ignore the four ways where three of the other four pitches
#are the strike/disengagement outcome, and the one way where all four are the
#strike/disengagement outcome, so we subtract another 5 * 15 which gives 150.
#Then we double it to get 300.
#Then for walks, we have 7 choose 4 which is also 35, and we subtract the 15 ways
#to get 4 in the first 6 which is 20. The other three pitches can have two outcomes
#so 20 * 2 * 2 * 2 which is 160, but we have to ignore the two scenarios where
#all three of the other three pitches are either all strikes or all disengagements
#so that's 20 * 2 which gives 120. So in total we need to subtract
# 420 * (n+1)*n**6 after multiplying by 3 and before multiplying by n again
stage8 = stage7 + (((stage7-stage6) * 3 - (420 * (n+1)*n**6)) * n)


#and that's it. The game can't go longer than 8 stages without ending, so after
#the 8th stage we're guarunteed to end because at worst you have two disengagemments,
#2 strikes, and 3 balls which is 7, so after decision 8 you're done. And when
#I set n = 1 these formulas matched the number of info states that openspiel got.
#When I set n = 2, the formulas matched what openspiel got. For n = 3
#my calculations matched what openspiel got, but it was slow so I'm going to
#stop there with the openspiel comps. I think I have the formula right.

In [6]:
stage1, stage2, stage3, stage4, stage5, stage6, stage7, stage8

(4, 40, 364, 3064, 23152, 149512, 761872, 2598952)

In [8]:
#experimenting with other n values
n = 21
stage1 = n+1
stage2 = stage1 + (stage1 * 3 * n)
stage3 = stage2 + ((stage2 - stage1) * 3 * n)
stage4 = stage3 + (((stage3 - stage2) * 3 - (2*(n+1)*n*n)) * n)
stage5 = stage4 + (((stage4 - stage3) * 3 - (13 * (n+1)*n*n*n)) * n)
stage6 = stage5 + (((stage5-stage4) * 3 - (56 * (n+1)*n*n*n*n)) * n)
stage7 = stage6 + (((stage6-stage5) * 3 - (180 * (n+1)*n**5)) * n)
stage8 = stage7 + (((stage7-stage6) * 3 - (420 * (n+1)*n**6)) * n)
stage8

8729219521660

In [11]:
stage8/1e12

8.72921952166

n = 1, 2, 3 give reasonably sized games with n = 3 being in the millions of info sets. But I think 3 lead distances is pretty low for a baserunner? I don't know I feel like you'd want more options than that. But maybe not honestly that could be like you have a short lead, a normal lead, and a big lead, so maybe 3 is perfectly reasonable. It'll be interesting to see as I start to solve for approximate equilibria how many lead distances the baserunner is actually choosing between. Is he mixing between all of the options or does he actually end up choosing between like 2 or 3 options. If it were up to me, based on the statcast range of lead distances, I'd at least want 8-14 feet, so 7 options, which apparently gives billions of info states (1e9). Billions would be tough to do in memory with vanilla cfr, but not impossible I don't think. But that would take a lot of effort and thought to efficiently store that in memory and to efficiently run those iterations, and then even after that you'd probably have to run iterations for days, so I think at that point we could justify going away from vanilla cfr. But then really we should include some values above and below 8 and 14 feet because if you average is 14 or 8 then sometimes you will be lower than that and sometimes higher. So let's say we did 6 to 16 feet, which is 11 options that results in tens of billions of info states (1e10). And foot long increments are pretty big, so if we did half a foot increments, that's 22 options which gives tens of trillions of info states (1e14), so now we are bigger than heads up limit holdem. And then finally, if we did the values that Scott Powers did, which is 0 to 20 feet in 0.1 step increments, so n = 200, we get 1e20, which is bigger than any solved game.

So my vote I think would be 6 to 16 feet if not just 0 to 20 feet broken down by feet, which will give either 1e10 info sets if we do 6 to 16 or 1e13 if we do 0 to 20, and 1e13 is exactly the same size as heads up limit holdem, so it is quite large. All that's to say, it is in our best interest to do the imperfect recall version of the game. In that version, we have an info set for the 12 ball/strike counts, 3 disengagement counts (0, 1, 2), and let's say 20 lead distances, which would come out to 20 X 12 X 3 = 720, so we go from 1e13 info sets to 720 lol. And, Lanctot's thesis suggests that we can run vanilla CFR on that version of the game and still minimize regret.  My concern though is even though we save a ton on memory, I'm not sure we save on runtime because we're still traversing every possible history. I wonder if you could do it in stages though. So stage 1 you start with a 3-2 count with two disengagements and you just run cfr to get the optimal strategy in that subgame. Then in stage 2, which is really stage 7, you replace the nonterminal children nodes with the values from the nodes you just got in stage 1 and turn them into terminal nodes, so then you can solve stage 2 as quickly as you solved stage 1 except that there are more options. You could have 3-2 with 1 disengagment or 2-2 with 2 disengagments or 3-1 with 2 disengeagments. Then you continue that process, so stage 3 which is really stage 6 you could have 3-2 with no disengagements, 2-2 with 1 disengagement, 1-2 with 2 disengagments, 2-1 with 2 disengagments, 3-0 with 2 disengamgents, etc. I think with this approach, each stage is a managably sized game that you can solve, and even though there are multiple root nodes in each stage, there would never be a crazy large number of root nodes, so you could easily parallelize over the roots.

I need to really read into the Lanctot phd disseration on imperfect recall games to make sure that the benefit of imperfect recall is only in memory and not in runtime. You would still have to visit every possible history. But if that is the case then I think the stage by stage approach will probably help because solving one stage at a time is much more managable. In fact, you may not even need to do cfr you may just be able to do like a normal form solver. Yeah as I'm writing this I have an idea, you totally could do a normal form solver on each stage. I think they'll be small enough, and in fact you can do better than the normal form you can do the behavioral strategy form that we did in the pitch sequencing paper where you have |A| columns for each info set. So in our case the pitcher can choose to pitch or pickoff, so if there are lets say 20 lead distances and therefore 20 info sets, the pitcher would have 40 rows or columns, 2 for each info set representing their behavioral strategy in each info set. Then you solve as you did in the pitch sequencing paper, but you start with the stage 8 game because then in the stage 7 games you can fill in the values of the nonterminal children nodes with the stage 8 game value, so you just have to work backward through the stages and then you will have an exact equilibrium instead of an epsilon-equilibrium, and my guess is it'll take less time than cfr would take. And I think you would only have to do that 36 times, once for each combination of ball/strike/num disengagements because we aren't doing the perfect recall version of the game, so you're just doing that simple linear program 36 times, and you can even do some in parallel, and then you're done!

Once you have that coded up and working I think you're ready to reach out to Scott Powers. You also may want to consider incorporating the pitch strategy and the batter swing strategy into the game, which would obviously make things more cmoplicated and may end up requiring some cfr type of algorithm. I don't necessarily think we need to do that I think the simpler version is more than good enough, but it may be interesting nonetheless. It's just not very practical from a baseball perspective because you probably wouldn't be able to solve for strategies quickly enough to use in game, but maybe you would if you ignore the prior pitch embedding that optimuspitch uses, which we will ahve to do anyways most likely to keep the state space reasonably sized.

In [ ]:
#in the case where the batter has only one lead distance to choose from,
#if you end the game after any pitch, there are 6 information sets. After two pitches,
#there are 30 info sets. After three pitches, there are 110. After four there are
#320. After 5 there are 740, and then after 6 which is the max number of pitches
#anyways, we get 1300

#if the batter has four actions, then the 6 pitches have 137145 info sets,
#5 pitches have 44025 info sets, four pitches have 9945 info sets, 3 pitches
#have 1713 info sets, 2 pitches have 225, and 1 pitch has 21

#If I do the lead distances of 6 to 15 feet, then one pitch has 1,221 info sets,
#two pitches has 71,841 info sets, 3 pitches has 2,848,241. When I went to
#4 pitches it ran for four hours trying to define the cfr_solver before I just
#ended it. So then I tried removing the pickoff action so the pitcher can
#only choose to pitch and the batter can choose a lead from 6 to 15 feet
#and to stay or go. The game that ended after 1 pitch had 11 info sets, which
#is the root plus the 10 batter leads. After 2 pitches we had 231 info sets.
#After 3 pitches, we had 4631 info sets. So I'm finally getting how to
#calculate this. In the following line of code, you can see how it works.
#I start with 11 info sets which is the root plus the ten lead distances.
#That's for 1 pitch. Then when I go to two pitches I have to add on the unique
#number of two pitch histories, all of which have a 1 pitch history as a prefix (11)
#and then two possible pitch outcomes that don't end the game ball or strike
#then 10 more possible batter leads, so I have to add 11 * 2 * 10.
#Then similarly for the third pitch, I have a two pitch history as a prefix
#so 11 * 2 * 10, and then I can have 2 non terminal pitch outcomes and
#10 more lead distances so another 2 * 10. So this approach works up through
#three pitches, but then I run into an issue. Any three pitch sequence that
#resulted in three strikes ends the at bat and doesn't have any children, so I need to
#subtract the children where I get a strike on the third pitch before multiplying by
#the ten possible lead distances on the fourth pitch. There are obviously
#11*2*10*2*10 * 2 three pitch histories, but I want to ignore the ball outcome
#to get the three strike histories, so that's 11 * 1 * 10 * 1* 10 * 1= 11*10*10,
#so I need to subtract 11 * 10 * 10 from 11*2*10*2*10*2 before multiplying
#by the 10 new lead distances after four pitches, and that does give the correct
#number of info sets. For the five pitch sequence, I need to do something similar
#only now I need to also subtract the third strike sequences and the
#four ball sequences, so I'm going to take the number I got for four pitch sequences,
#multiply by 2 for the two possible pitch outcomes, then I need to subtract
#the number of ways you can get two strikes or three balls in four pitches (there are
#4 ways to do those things) times 11*10*10*10, so subtract 4*11*10*10*10,
#then finally multiply by 10 again to get the new lead distances. That gives
#the correct number of info sets.
#Then finally for six pitch sequence, I'm going to take my five pitch sequence number
#and multiply by 10 for the possible lead distances then that's it because at that point
#the ab is guaranteed to be over, and that gave the correct number so this formula seems to work for pitches
11 + (11 * 2 * 10) + (11 * 2 * 10 * 2 * 10) + (((11*2*10*2*10*2)-11*10*10) * 10) + ((((((11*2*10*2*10*2)-11*10*10) * 10)*2) - 4*11*10*10*10) * 10) + ((((((11*2*10*2*10*2)-11*10*10) * 10)*2) - 4*11*10*10*10) * 10) *10

12181631

I looked up the average distance from the base at pitchers first move for MLB players on all stolen base opportunities https://baseballsavant.mlb.com/leaderboard/basestealing-run-value?game_type=Regular&n=q&pitch_hand=all&runner_moved=All&target_base=All&prior_pk=All&season_end=2025&season_start=2025&sortColumn=r_primary_lead&sortDirection=desc&split=no&team=&type=Bat&with_team_only=1&expanded=1, and the values ranged from 14.2 feet to 8.2 feet, so I then tried allowing lead distances from 6 to 15 feet. That ran for like 3-4 hours before crashing so I never saw how many info sets there were.